<a href="https://colab.research.google.com/github/Junoflows/Hands_on_ML/blob/main/Chapter_11_%EC%8B%AC%EC%B8%B5_%EC%8B%A0%EA%B2%BD%EB%A7%9D_%ED%9B%88%EB%A0%A8%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 설정

In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Chapter 11 심층 신경망 훈련하기

심층 신경망
+ 수백 개의 뉴런을 구성된 10개 이상의 층을 사용하는 신경망

심층 신경망 훈련 중 발생하는 문제
+ 그레디언트 소실 / 폭주 문제
+ 훈련 데이터 부족 또는 너무 비싼 레이블 작업
+ 극단적으로 느린 훈련과정
+ 과대적합 - 특히 훈련 샘플이 충분하지 않거나 잡음이 많은 경우

이 장에서는 위 문제들의 해결책 제시한다.

## 11.1 그레디언트 소실과 폭주 문제

역전파 알고리즘
+ 출력층에서 입력층으로 오차 그레디언트를 전파
+ 하위층으로 갈수록 그레디언트 소실/폭주 문제가 발생함

원인 : 활성화 함수와 가중치 초기화를 위해 아래 조합을 선택하였기 때문
+ 활성화 함수 : 시그모이드 함수 사용
+ 가중치 초기화 : 표준정규분포 활용
+ 위 방식을 사용했을 때 각 층에서 출력의 분산이 입력의 분산보다 더 크다는 것을 밝혀짐

<img src = 'https://formal.hknu.ac.kr/handson-ml2/slides/images/ch11/homl11-01.png' width = 50%> <br/>
+ 입력이 커지면 기울기가 0에 수렴하므로 하위 증으로 갈수록 그레디언트가 전파되지 않는다.

### 11.1.1 글로럿, He 초기화

#### 글로럿 초기화
+ 각 층의 출력에 대한 분산과 입력에 대한 분산과 같아야 함
+ 역방향에서 층을 통과하기 전과 후의 그레디언트 분산이 같아야 함
+ 각 층의 연결 가중치를 아래 방식대로 무작위로 초기화하는 대안을 제안

팬-인/팬-아웃
+ $fan_{in}$ : 층에 들어오는 입력 수
+ $fan_{out}$ : 층에서 나가는 출력 수
+ $ fan_{avg} = \frac{fan_{in} + fan_{out} }{2} $


글로럿 초기화 (시그모이드 활성화 함수를 사용할 때)
+ 평균이 0 이고 분산 $σ^2 = \frac{1}{fan_{avg}}$ 인 정규분포로 초기화
+ $r = \sqrt{\frac{3}{fan_{avg}}}$ 이고 $-r$과 $r$ 사이의 균등분포로 초기화

르쿤(LeCun) 초기화
+ 글로럿 초기화에서 $fan_{avg}$ 를 $fan_{in}$ 으로 대체

#### He 초기화
+ ReLU 계열의 활성화 함수를 사용했을 때의 사용하는 초기화 전략
+ $σ^2 = \frac{2}{fan_{in}}$ (정규분포 활용 초기화)

+ $r = \sqrt{3σ^2}$ (균등분포 활용 초기화)

#### 활성화 함수와 초기화 방식

| 초기화 전략      | 활성화 함수                                       | 정규분포 초기화 |
|------------------|--------------------------------------------------|----------------|
| Glorot 초기화    | 활성화 함수 없는 경우, 하이퍼볼릭 탄젠트, 로지스틱, 소프트맥스 | glorot_normal  |
| He 초기화        | ReLU 함수와 그 변종들                             | he_normal      |
| LeCun 초기화     | SELU                                             | lecun_normal   |


+ 케라스는 기본적으로 균등분포의 글로럿 초기화를 사용
+ 정규분포를 이용한 He 초기화를 사용하고자 하는 경우

In [ ]:
import tensorflow as tf
from tensorflow import keras

keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

+ $fan_{in}$ 대신 $fan_{out}$  기반의 균등분포 He 초기화를 사용하고자 할 경우

In [ ]:
# VarianceScaling 클래스 활용
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)

### 11.1.2 수렴하지 않는 활성화 함수

+ 심층신경망의 층에서 사용되는 활성화 함수는 시그모이드 함수보다 아래 함수들이 보다 좋은 성능을 발휘함
+ ReLU, LeakyReLU, RReLU, PReLU, ELU, SELU

#### ReLU
+ 특정 양수 값에 수렴하지 않고 계산도 빠름
+ $ReLU(z) = max(0, z)$
+ __죽은 ReLU__ 로 알려진 문제가 존재
  + 입력 가중치 합이 음수가 되면 뉴런이 죽게 되어 경사하강법이 제대로 작동하지 않음

<img src = 'https://formal.hknu.ac.kr/handson-ml2/slides/images/ch11/homl11-02a.png' width = 50%>

#### LeakyReLU
+ $LeakyReLU(z) = max(az, z)$
  + $a$ : 새는(leaky) 정도를 결정하는 하이퍼파라미터
+ ReLU 보다 좋은 성능 발휘
  + 일반적으로 $a$는 0.01이나 0.2를 많이 사용하고 기본값은 0.01

<img src = 'https://formal.hknu.ac.kr/handson-ml2/slides/images/ch11/homl11-03.png' width = 50%>

#### RReLU
+ $a$를 주어진 범위에서 무작위로 선택하는 LeakyReLU
+ 잘 작동하며 과대적합을 줄이는 규제역할도 수

#### PReLU
+ 역전파 과정에서 $a$값도 자동 조정됨(하이퍼파라미터가 아님)
+ 대규모 데이터셋에서 ReLU보다 성능이 앞섰지만 소규모 데이터셋에서는 과대적합 위험성이 존재함

#### ELU
+ ReLU 계열 활성화함수들보다 훈련 시간이 줄고 성능이 더 높음
+ $ \text{ELU}_\alpha(z) = \begin{cases}
  \alpha (\exp(z) - 1) & \text{if } z < 0 \\
  z & \text{if } z \geq 0
\end{cases} $


<img src = 'https://formal.hknu.ac.kr/handson-ml2/slides/images/ch11/homl11-04.png' width = 50%>

ELU 함수의 장단점
+ 수렴 속도 빠름
+ 지수함수가 사용되어 계산이 느림
+ 테스트 시 ReLU 를 사용한 신경망보다 느림

#### SELU
+ 스케일이 조정된 ELU 함수의 변종
+ 아래 조건에서 뛰어난 성능을 보임
  + 입력 특성이 표준화되어야 함
  + 모든 은닉층에서 르쿤 정규분포 초기화 사용
  + 일렬로 쌓은 층으로 구성되어야 함
  + 모든 층이 완전연결층이어야 함

#### GELU

+ $ \text{GELU}(x) = x \cdot \Phi(x)$ <br/>
+ $ \Phi(x) = \frac{1}{2} \left[1 + \text{erf}\left(\frac{x}{\sqrt{2}}\right)\right] $



<img src = 'https://iq.opengenus.org/content/images/2021/12/gelufunction.png' width = 50%>

+ 입력 데이터의 가우시안 분포를 가정
+ 자연어 처리 분야에서 널리 사용

#### 심층신경망의 은닉층에 대한 활성화 함수 선택 가이드라인
+ 일반적인 우선순위 : SELU > ELU > LeakyReLU 기반 > ReLU > 시그모이드
+ 신경망이 Self-normalizing 하지 않은 경우 : SeLU 보다 ELU 선호
+ 시간과 컴퓨팅 파워가 충분하다면 교차검증을 통해 여러 활성화 함수 성능을 비교
+ 실행속도가 중요한 경우: LeakyReLU
+ 과대적합 발생하는 경우: RReLU
+ 훈련세트가 매우 큰 경우: PReLU
+ 훈련속도가 중요한 경우: ReLU

#### 예시
+ RReLU 를 제외한 활성화함수는 쉽게 구현가능

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 300)               235500    
                                                                 
 p_re_lu (PReLU)             (None, 300)               300       
                                                                 
 dense_3 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266910 (1.02 MB)
Trainable params: 266910 

### 11.1.3 배치정규화

+ ELU(또는 다른 ReLU 변종) + He 초기화 를 사용하면 훈련 초기 그레디언트 소실/폭주 문제 해결
+ 훈련 중 동일 문제가 재발하지 않을 것이란 보장이 없음
+ 해결 방안으로 배치정규화 기법을 제안

#### 배치정규화 알고리즘

1. $\mu_B = \frac{1}{m_B} \sum_{i=1}^{m_B} x^{(i)}$

2. $\sigma_B^2 = \frac{1}{m_B} \sum_{i=1}^{m_B} (x^{(i)} - \mu_B)^2$

3. $\hat{x}^{(i)} = \frac{x^{(i)} - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$ ($\epsilon$ 은 일반적으로 $10^{-5}$)

4. $z^{(i)} = \gamma \odot \hat{x}^{(i)} + \beta$ ($\beta$ : 층의 출력 이동 파라미터)


+ 훈련 시 입력을 정규화하고 스케일을 조정하고 이동시킴

#### 배치 정규화의 효과
+ 드롭아웃 같은 규제 역할을 하여 다른 규제 기법의 필요성을 줄여줌
+ sigmoid 나 tanh 활성화 함수 사용 가능


#### 케라스로 배치정규화 구현
+ 은닉층의 활성화 함수 이전이나 이후에 BatchNormalization 층 추가

In [ ]:
# 활성화 함수 이후에 정규화 층 추가
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
# 활성화 함수 이전에 정규화 층 추가
# 활성화 함수를 정규화 층 뒤에 별도의 층으로 추가함
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

#### 배치정규화 활용
+ 보통 모든 층 뒤에 배치정규화가 있다고 가정
+ 따라서 신경망 그림에 종종 생략됨

## 11.2 전이학습

#### 전이학습
+ 비슷한 기능을 가진 사전훈련된 모델의 일부 층을 재사용하는 방법
+ 훈련속도를 크게 높이고 필요한 훈련 데이터의 양도 크게 줄여줌
+ 하위층에서 저수준 특성이 학습되기 때문에 비슷한 원본모델의 하위 은닉층이 훨씬 유용함

<img src = 'https://formal.hknu.ac.kr/handson-ml2/slides/images/ch11/homl11-05.png' width = 50%>

전이학습 단계
1. 재사용 층을 모두 동결하고 훈련
2. 성능 평가 후 1~2개 은닉층의 동결을 해제하고 가중치 조정 - 학습률을 줄이는게 가중치 조정에 효과적
3. 성능이 좋아지지 않거나 데이터가 적을 경우 상위 은닉층 제거 후 남은 은닉층 동결하고 다시 훈련
4. 훈련 데이터가 많을 경우 더 많은 은닉층 추가 가능
5. 위 과정 반복

### 11.2.1 케라스를 사용한 전이 학습

가정 : model_A 주어짐
  + 샌들과 셔츠를 제외한 8개의 클래스만 담겨있는 패션 MNIST
  + 90% 이상의 정확도 성능을 갖는 8개의 클래스 분류 학습 모델

목표 : 셔츠와 샌들을 분류하는 이진분류기 model_B 훈련

방법 : 전이학습을 이용한 model_B_on_A 훈련

+ model_A 생성

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 13s 8ms/step - loss: 0.5734 - accuracy: 0.8183 - val_loss: 0.3859 - val_accuracy: 0.8632
Epoch 2/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3559 - accuracy: 0.8781 - val_loss: 0.3243 - val_accuracy: 0.8871
Epoch 3/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.3197 - accuracy: 0.8904 - val_loss: 0.2997 - val_accuracy: 0.8961
Epoch 4/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3001 - accuracy: 0.8972 - val_loss: 0.2867 - val_accuracy: 0.9031
Epoch 5/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.2870 - accuracy: 0.9021 - val_loss: 0.2763 - val_accuracy: 0.9061
Epoch 6/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2772 - accuracy: 0.9059 - val_loss: 0.2724 - val_accuracy: 0.9066
Epoch 7/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.2689 - accuracy: 0.9094 - val_loss: 0.2678 - val_accuracy

In [ ]:
model_A.save("my_model_A.h5")

+ 전이학습하지 않는 model_B 생성

In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 51ms/step - loss: 0.6935 - accuracy: 0.5750 - val_loss: 0.6110 - val_accuracy: 0.6410
Epoch 2/20
7/7 [==============================] - 0s 18ms/step - loss: 0.5670 - accuracy: 0.7000 - val_loss: 0.5068 - val_accuracy: 0.7667
Epoch 3/20
7/7 [==============================] - 0s 19ms/step - loss: 0.4673 - accuracy: 0.7950 - val_loss: 0.4333 - val_accuracy: 0.8448
Epoch 4/20
7/7 [==============================] - 0s 32ms/step - loss: 0.3965 - accuracy: 0.8550 - val_loss: 0.3774 - val_accuracy: 0.8925
Epoch 5/20
7/7 [==============================] - 0s 19ms/step - loss: 0.3420 - accuracy: 0.9200 - val_loss: 0.3318 - val_accuracy: 0.9280
Epoch 6/20
7/7 [==============================] - 0s 32ms/step - loss: 0.2964 - accuracy: 0.9500 - val_loss: 0.2970 - val_accuracy: 0.9361
Epoch 7/20
7/7 [==============================] - 0s 32ms/step - loss: 0.2633 - accuracy: 0.9700 - val_loss: 0.2697 - val_accuracy: 0.9462
Epoch 8/20
7/7 [===========

+ 전이학습 이용하는 model_B_on_A 생성

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

+ model_B_on_A와 model_A는 층을 공유하기 때문에 하나를 훈련하면 두 모델이 업데이트됨
+ 따라서 model_A를 클론한 것을 사용해 model_B_on_A를 만들어야 함

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 63ms/step - loss: 0.4382 - accuracy: 0.7800 - val_loss: 0.4661 - val_accuracy: 0.7688
Epoch 2/4
7/7 [==============================] - 0s 30ms/step - loss: 0.4160 - accuracy: 0.7800 - val_loss: 0.4425 - val_accuracy: 0.7819
Epoch 3/4
7/7 [==============================] - 0s 18ms/step - loss: 0.3935 - accuracy: 0.7950 - val_loss: 0.4219 - val_accuracy: 0.7941
Epoch 4/4
7/7 [==============================] - 0s 31ms/step - loss: 0.3738 - accuracy: 0.7950 - val_loss: 0.4027 - val_accuracy: 0.8012
Epoch 1/16
7/7 [==============================] - 1s 64ms/step - loss: 0.3063 - accuracy: 0.8450 - val_loss: 0.2774 - val_accuracy: 0.8945
Epoch 2/16
7/7 [==============================] - 0s 19ms/step - loss: 0.2091 - accuracy: 0.9250 - val_loss: 0.2113 - val_accuracy: 0.9300
Epoch 3/16
7/7 [==============================] - 0s 18ms/step - loss: 0.1565 - accuracy: 0.9700 - val_loss: 0.1735 - val_accuracy: 0.9533
Epoch 4/16
7/7 [===============

model_B 와 model_B_on_A 성능 비교

In [ ]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 7ms/step - loss: 0.1244 - accuracy: 0.9840


[0.1243593692779541, 0.984000027179718]

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 6ms/step - loss: 0.0578 - accuracy: 0.9900


[0.05782657116651535, 0.9900000095367432]

In [1]:
(100 - 98.40) / (100 - 99)

1.5999999999999943

### 11.2.2 비지도 전이학습

+ 레이블된 훈련 데이터가 없을 경우
+ 레이블이 없는 훈련 데이터를 오토인코더나 GAN 등을 이용해 레이블 지정 후 하위층을 재사용

<img src = 'https://velog.velcdn.com/images/tedlim23/post/46517f98-e66a-4ccb-b44a-66b1ad08e4df/Untitled%201.png' width = 50%>

### 11.2.3 보조 작업에서 전이학습

+ 레이블된 훈련 데이터가 적을 경우
+ 레이블된 데이터를 얻거나 생성할 수 있는 보조 작업에서 첫 번째 신경망을 훈련하고 재사용

예제 : 얼굴 인식 시스템
+ 개인별 이미지가 많지 않은 경우
+ 인터넷에서 무작위로 인물 이미지 수집
+ 두 개의 다른이미지를 분류하는 신경망 훈련
+ 학습된 모델의 하위층을 재사용
+ 적은 양의 레이블된 훈련 데이터를 이용하여 얼굴 인식 분류기 학습 가능

## 11.3 고속 옵티마이저

훈련 속도를 크게 높일 수 있는 옵티마이저 소개

경사 하강법 알고리즘
+ $\theta = \theta - \eta \cdot \nabla_{\theta} J(\theta)$

### 11.3.1 모멘텀 최적화

1. $m \leftarrow \beta m - \eta \nabla_{\theta} J(\theta)$
2. $\theta \leftarrow \theta + m$


+ 이전 그레디언트가 이후 그레디언트에 영향을 줌
+ 일반 경사하강법보다 빠르게 전역 최소점에 도달

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

### 11.3.2 Nesterov 가속 경사(NAG)

1. $m \leftarrow \beta m - \eta \nabla_{\theta} J(\theta + \beta m)$
2. $\theta \leftarrow \theta + m$


+ 모멘텀 가속화 기법을 수정
+ 기존 모멘텀 최적화보다 훈련속도가 빠름

<img src = 'https://formal.hknu.ac.kr/handson-ml2/slides/images/ch11/homl11-06.png' width = 50%>

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

### 11.3.3 AdaGrad

1. $s \leftarrow s + \nabla_{\theta}J(\theta) \otimes \nabla_{\theta}J(\theta)$
2. $\theta \leftarrow \theta - \eta \nabla_{\theta}J(\theta) \oslash \sqrt{s + \epsilon}$


+ 전역 최적점 방향으로 더 곧장 가도록 도와줌
+ 간단한 2차방정식 문제에 대해서 잘 작동함
+ 신경망 훈련 시 너무 일찍 멈추는 경향이 있음

<img src = 'https://formal.hknu.ac.kr/handson-ml2/slides/images/ch11/homl11-07.png' width = 50%>

In [ ]:
optimizer = keras.optimizers.Adagrad(lr=0.001)

### 11.3.4 RMSProp

1. $s \leftarrow \beta s + (1 - \beta) \nabla_{\theta}J(\theta) \otimes \nabla_{\theta}J(\theta)$
2. $\theta \leftarrow \theta - \eta \nabla_{\theta}J(\theta) \oslash (\sqrt{s + \epsilon})$


+ AdaGrad의 이른 종료 문제점을 해결한 기법
+ 가장 최근 반복의 그레디언트만 누적하여 해결 (지수 감소 평균)

### 11.3.5 Adam과 Nadam 최적화

1. $m \leftarrow \beta_1 m - (1 - \beta_1) \nabla_{\theta}J(\theta)$
2. $s \leftarrow \beta_2 s + (1 - \beta_2) \nabla_{\theta}J(\theta) \otimes \nabla_{\theta}J(\theta)$
3. $\hat{m} \leftarrow \frac{m}{1 - \beta_1^t}$
4. $\hat{s} \leftarrow \frac{s}{1 - \beta_2^t}$
5. $\theta \leftarrow \theta + \eta \hat{m} \oslash (\sqrt{\hat{s} + \epsilon})$


+ 모멘텀 최적화와 RMSProp 아이디어를 활용
+ 3,4 단계에서 m, s 를 0으로 치우치지 않도록 증폭하는 효과

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

#### AdaMax

1. $m \leftarrow \beta_1 m - (1 - \beta_1) \nabla_{\theta}J(\theta)$
2. $s \leftarrow \max(\beta_2 s, \nabla_{\theta}J(\theta))$
3. $\hat{m} \leftarrow \frac{m}{1 - \beta_1^t}$
4. $\theta \leftarrow \theta + \eta \hat{m} \oslash (\sqrt{\hat{s} + \epsilon})$


+ Adam 알고리즘 개선
+ 경우에 따라 Adam 성능이 더 좋음

In [ ]:
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

#### Nadam

+ Adam + Nesterov
+ 일반적으로 Adam 보다 성능 좋지만 경우에 따라 RMSProp이 더 좋기도 함


In [ ]:
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

#### 옵티마이저 정리

+ 선택한 옵티마이저의 성능이 만족스럽지 않을 경우 기본 Nesterov 가속 경사 사용 추천
+ 새로운 기법 활용에 관심 가질 것

| 클래스                             | 수렴 속도 | 수렴 품질           |
|------------------------------------|-----------|---------------------|
| SGD                                | *         | ***                 |
| SGD(momentum=...)                  | **        | ***                 |
| SGD(momentum=..., nesterov=...)    | **        | ***                 |
| Adagrad                            | ***       | * (너무 일찍 멈춤)   |
| RMSProp                            | ***       | ** 또는 ***         |
| Adam                               | ***       | ** 또는 ***         |
| Nadam                              | ***       | ** 또는 ***         |
| AdaMax                             | ***       | ** 또는 ***         |


## 11.4 규제를 사용해 과대적합 피하기

+ 심층 신경망에 사용되는 수백만 개의 파라미터에 의해 과대적합이 발생하기 쉬움
+ 다양한 규제를 사용해 과대적합을 해결

지금까지의 살펴본 규제 기법
  + EarlyStopping, 배치정규화

### $l_1$, $l_2$ 규제

#### $l_1$ 규제
+ $ L = L_{\text{original}} + \lambda \sum_{i=1}^{n} |w_i|$
+ 덜 중요한 특성의 가중치를 0으로 만들어서 학습에서 제외시키는 효과 (특성 선택)

#### $l_2$ 규제
+ $L = L_{\text{original}} + \lambda \sum_{i=1}^{n} w_i^2$
+ 가중치의 크기를 줄이되 0으로 만들지는 않음
+ 가중치의 큰 값에 대해 더 큰 패널티를 부여

In [ ]:
layer = keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

### 11.4.2 드롭 아웃

+ 신경망에서 가장 인기 있는 규제기법
+ 일반적으로 매우 잘 작동함

#### 아이디어

<img src = 'https://formal.hknu.ac.kr/handson-ml2/slides/images/ch11/homl11-09.png' width = 50%>

+ 매 훈련에서 각 뉴런을 특정 확률로 훈련에서 제외시킴
+ 드롭아웃 비율을 올려야 하는 경우
  + 과대적합 발생하는 경우
  + 층에 많은 뉴런이 포함될 때

+ 일반적으로 출력층을 제외한 최상위 3개 층에 대해 드롭아웃 적용

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
    ])

### 11.4.3 몬테 카를로 드롭아웃

+ 훈련된 드롭아웃 모델을 재훈련하지 않으면서 성능을 향상시키는 기법
+ 모델의 불확실성을 더 잘 측정할 수 있음

### 11.4.4 맥스-노름(max-norm) 규제

+ 각 뉴런에 대해 입력 가중치 $w$가 $||w|| \le r$ 이 되도록 제한
+ 14장 CNN 층에서 max-norm 을 적용하려면 axis 값을 적절히 지정해야함 (axis = [0,1,2])

In [ ]:
layer = keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal",
                           kernel_constraint=keras.constraints.max_norm(1.))

## 11.5 요약 및 실용적인 가이드 라인

#### 기본 DNN 설정
| 하이퍼파라미터      | 기본값                             |
|-------------------|----------------------------------|
| 커널초기화          | He 초기화                          |
| 활성화 함수         | ELU                               |
| 정규화              | 깊은 신경망일 경우에만 배치정규화 사용     |
| 규제                | 조기종료, 경우에 따라 $ l_2$ 규제 추가 |
| 옵티마이저           | 모멘텀 최적화, RMSProp, Nadam 중 하나    |
| 학습률 스케줄        | 1사이클                            |



#### 자기정규화를 위한 DNN 설정
| 하이퍼파라미터      | 기본값                                         |
|-------------------|-----------------------------------------------|
| 커널초기화         | 르쿤(LeCun) 초기화                             |
| 활성화 함수        | SELU                                           |
| 정규화             | 필요 없음                                      |
| 규제               | 경우에 따라 알파 드롭아웃|
| 옵티마이저          | 모멘텀 최적화, RMSProp, Nadam 중 하나         |
| 학습률 스케줄       | 1사이클                                        |

#### 희소모델인 경우
+ $l_1$ 규제 사용 추천
+ 매우 희소한 모델이 필요한 경우
  + 텐서플로우의 모델최적화 툴킷(TF-MOT) 사용 가능

#### 빠른 응답 모델인 경우
+ 은닉층 수 줄이기
+ 배치 정규화 층을 이전 층과 합치기
+ LeakyReLU 또는 ReLU 사용
+ 희소모델 사용

#### 예측속도보다 정확도에 충실한 모델인 경우
+ 몬테카를로 드롭아웃 사용 추천